In [1]:
#Import all the required libraries
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
#read the data
df=pd.read_csv('news.csv')

In [3]:
#get shape and head of data
df.shape
df.head(5)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
#get the labels
labels=df.label
labels.head(5)

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [5]:
#split the data into training data and testing data
x_train, x_test, y_train, y_test = train_test_split(df['text'],labels,test_size=0.2, random_state=0)

In [6]:
#fit and transform train set and transform test set
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)
tfid_train = tfidf_vectorizer.fit_transform(x_train)
tfid_test = tfidf_vectorizer.transform(x_test)

In [8]:
#predict on the test set and calculate the accuracy
pac=PassiveAggressiveClassifier(max_iter=30)
pac.fit(tfid_train,y_train)
y_pred=pac.predict(tfid_test)
score=accuracy_score(y_test,y_pred)
print(f'Accuracy: {round(score*100,2)}%')

Accuracy: 93.21%


In [9]:
confusion_matrix(y_test,y_pred,labels=['FAKE','REAL'])

array([[572,  43],
       [ 43, 609]], dtype=int64)

In [10]:
def fake_news_det(news):
    input_data = [news]
    vectorized_input_data = tfidf_vectorizer.transform(input_data)
    prediction = pac.predict(vectorized_input_data)
    print(prediction)

In [11]:
fake_news_det('The shooting, she said, was cowardly. The gunman “sat amongst them.”')

['REAL']


In [13]:
import pickle
pickle.dump(pac,open('model.pkl', 'wb'))

In [14]:
# load the model from disk
loaded_model = pickle.load(open('model.pkl', 'rb'))

In [20]:
def fake_news_det1(news):
    input_data = [news]
    vectorized_input_data = tfidf_vectorizer.transform(input_data)
    prediction = loaded_model.predict(vectorized_input_data)
    print(prediction)

In [21]:
fake_news_det1('The church was burned down by white crowds. “And even more strict regulations were imposed on Charleston’s Black Churches,” the book said. By 1834, all black churches were closed by state law. Some members of the church joined white churches, and “others continued the tradition of the African church by worshiping underground.” In 1865, Emanuel AME resurfaced with 3,000 members.')

['REAL']
